# 🚀 Proyecto ELT con SpaceX API  
## Notebook 01 – Extracción y Carga

En este notebook se realiza la **extracción de datos desde la API pública de SpaceX** y el **almacenamiento en Delta Lake**.  

Se siguen los pasos de la consigna:  
1. Extracción de **2 o más endpoints**.  
2. Uso de al menos un **endpoint dinámico (actualizable)** y otro **estático**.  
3. Guardado en **formato Delta Lake**.  
4. Aplicación de **extracción incremental y full** según corresponda.  


In [ ]:
# =========================
# CONFIGURACIÓN INICIAL
# =========================

import sys

# Ajusto para que la reconozca la ruta del proyecto
sys.path.append("..") 

# --- IMPORTS DEL PROYECTO ---
from src.extract import fetch_data
from src.load import save_to_delta
from src.utils import setup_logger

# --- LOGGER ---
logger = setup_logger()
logger.info("Inicio de ejecución del notebook")


2025-08-28 20:22:24,171 - INFO - Inicio de ejecución del notebook


## 🔗 Endpoints seleccionados

- **Dinámicos:**  
  - `launches/latest` → Último lanzamiento.  
  - `launches/upcoming` → Próximos lanzamientos.  

- **Estáticos:**  
  - `rockets` → Información de cohetes.  
  - `dragons` → Información de cápsulas Dragon.  

In [2]:
# Extracción de datos
df_latest = fetch_data("latest_launch")
df_upcoming = fetch_data("upcoming_launches")
df_rockets = fetch_data("rockets")
df_dragons = fetch_data("dragons")

logger.info(f"Último lanzamiento: {len(df_latest)} registros")
logger.info(f"Próximos lanzamientos: {len(df_upcoming)} registros")
logger.info(f"Cohetes: {len(df_rockets)} registros")
logger.info(f"Dragons: {len(df_dragons)} registros")

2025-08-28 20:22:26,751 - INFO - Último lanzamiento: 1 registros
2025-08-28 20:22:26,752 - INFO - Próximos lanzamientos: 18 registros
2025-08-28 20:22:26,754 - INFO - Cohetes: 4 registros
2025-08-28 20:22:26,754 - INFO - Dragons: 2 registros


## 💾 Guardado en Delta Lake

Se guarda cada DataFrame en formato **Delta Lake**:  
- Los **endpoints dinámicos** (`latest_launch`, `upcoming_launches`) se almacenan con **particiones por fecha (extracción incremental)**.  
- Los **endpoints estáticos** (`rockets`, `dragons`) se guardan en una única ruta (extracción full).  

In [5]:
try:
    save_to_delta(df_latest, "latest_launch", incremental=True)
    save_to_delta(df_upcoming, "upcoming_launches", incremental=True)
    save_to_delta(df_rockets, "rockets", incremental=False)
    save_to_delta(df_dragons, "dragons", incremental=False)

    logger.info("✅ Todos los datasets fueron guardados correctamente en Delta Lake.")

except Exception as e:
    logger.error(f"❌ Error guardando datasets en Delta Lake: {e}")
    raise

logger.info("Todos los datasets fueron guardados correctamente en Delta Lake.")


2025-08-28 20:23:14,983 - ERROR - ❌ Error guardando datasets en Delta Lake: An error occurred while calling o116.save.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: delta. Make sure the provider name is correct and the package is properly registered and compatible with your Spark version. SQLSTATE: 42K02
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:722)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:681)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:740)
	at org.apache.spark.sql.classic.DataFrameWriter.lookupV2Provider(DataFrameWriter.scala:626)
	at org.apache.spark.sql.classic.DataFrameWriter.saveInternal(DataFrameWriter.scala:135)
	at org.apache.spark.sql.classic.DataFrameWriter.save(DataFrameWriter.scala:118)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invok

Py4JJavaError: An error occurred while calling o116.save.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: delta. Make sure the provider name is correct and the package is properly registered and compatible with your Spark version. SQLSTATE: 42K02
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:722)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:681)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:740)
	at org.apache.spark.sql.classic.DataFrameWriter.lookupV2Provider(DataFrameWriter.scala:626)
	at org.apache.spark.sql.classic.DataFrameWriter.saveInternal(DataFrameWriter.scala:135)
	at org.apache.spark.sql.classic.DataFrameWriter.save(DataFrameWriter.scala:118)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.